libraries

In [2]:
import cv2 as cv
import os
import numpy as np
import math
import random
import shutil

pre trained models

In [3]:
models = cv.CascadeClassifier('./pre-trained-models/haarcascade_frontalface_default.xml')

dataset

In [5]:
dataset_path = './Dataset'

train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')

os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

train_ratio = 0.75

for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    
    if not os.path.isdir(folder_path):
        continue
    
    images = os.listdir(folder_path)
    
    # number of train images
    n = int(len(images) * train_ratio)
    
    # random shuffle
    random.shuffle(images)
    
    # SPLIT DATA
    train_img = images[:n]
    test_img = images[n:]
    
    # create separate folder in train folder
    train_folder = os.path.join(train_path, folder_name)
    os.makedirs(train_folder, exist_ok=True)
    
    # copy train images to train folder
    for img in train_img:
        source_path = os.path.join(folder_path, img)
        dest_path = os.path.join(train_folder, img)
        shutil.copy(source_path, dest_path)
        
    # copy to test images to test folder
    for img in test_img:
        source_path = os.path.join(folder_path, img)
        # rename test images just incase duplicate name
        dest_path = os.path.join(test_path, f'{folder_name}-{img}')
        shutil.copy(source_path, dest_path)

PermissionError: [Errno 13] Permission denied: './Dataset\\train\\romelu_lukaku'

In [6]:
# delete additional folders that will be made
shutil.rmtree('./Dataset/train/train')
shutil.rmtree('./Dataset/train/test')